In [4]:
import pandas as pd
from dateutil.parser import parse
import hashlib
import string
import re
import os

In [6]:
df = pd.read_csv('/Users/zixiang/Documents/tap_de23/section1/raw_datasets/dataset_1_9c7f5_20230805_01hour.csv')

In [7]:
df = df.dropna(subset=['applicant_name'], how='all', inplace=False)
df = df[df['applicant_name'].str.strip() != '']

In [8]:
salutations = ['Mr.', 'Mrs.', 'Ms.', 'Dr.', 'Miss.', 'Sir.', 'Madam.', 'Prof.', 'Rev.']

def remove_salutations(name):
    for salutation in salutations:
        name = name.replace(salutation, '')
    return name.strip()

df.loc[:, 'applicant_name'] = df['applicant_name'].apply(remove_salutations)

name_split = df['applicant_name'].str.split(n=1, expand=True)
trans = str.maketrans('', '', string.punctuation + ' ')
df.loc[:, 'first_name'] = name_split[0]
df.loc[:, 'last_name'] = name_split[1].apply(lambda x: x.translate(trans))

In [9]:
df = df[pd.to_numeric(df['applicant_mobile'], errors='coerce').notnull()]
df = df[df['applicant_mobile'].astype(str).str.len() == 8]

In [10]:
df = df[df['applicant_email'].str.endswith(('.com', '.net'))]

In [11]:
def format_date(date_str):
    try:
        date_obj = parse(date_str)
        return date_obj.strftime('%Y%m%d')
    except:
        return None

df.loc[:,'applicant_dob'] = df['applicant_dob'].apply(format_date)
df.loc[:,'date_of_birth'] = df['applicant_dob'].apply(format_date)
df = df.dropna(subset=['applicant_dob'], how='all', inplace=False)

In [12]:
df['applicant_dob'] = pd.to_datetime(df['applicant_dob'])
df['age'] = (pd.Timestamp.now() - df['applicant_dob']).dt.days//365
df['above_18'] = df['age'] >= 18

In [13]:
df.head()

,applicant_name,applicant_email,applicant_mobile,applicant_dob,first_name,last_name,date_of_birth,age,above_18
0,Danielle Daniels,DanielleDaniels@houston.com,23386434,1973-01-10,Danielle,Daniels,19730110,50,True
3,Evelyn Ware Jr.,EvelynWare@west-price.com,42435202,1972-12-23,Evelyn,WareJr,19721223,50,True
4,Matthew Clark PhD,MatthewClark@gardner-dickerson.com,23172514,2023-05-23,Matthew,ClarkPhD,20230523,0,False
5,Autumn Johnson,AutumnJohnson10@houston.com,56281039,1974-01-16,Autumn,Johnson,19740116,49,True
6,Jennifer Miller,JenniferMiller30@brown-greene.com,79733291,1988-08-14,Jennifer,Miller,19880814,34,True


In [136]:
successful_df = df[df['above_18']]
successful_df = successful_df[['first_name','last_name','date_of_birth','above_18']]

In [138]:
successful_df.loc[:,'Membership_ID'] = (successful_df['last_name'] + '_' + successful_df['date_of_birth'].apply(lambda dob: hashlib.sha256(dob.encode()).hexdigest()[:5]))

In [139]:
successful_df.head()

,first_name,last_name,date_of_birth,above_18,Membership_ID
0,Danielle,Daniels,19730110,True,Daniels_7a512
3,Evelyn,WareJr,19721223,True,WareJr_da04e
5,Autumn,Johnson,19740116,True,Johnson_18122
6,Jennifer,Miller,19880814,True,Miller_8e3b0
8,Douglas,Franklin,19930824,True,Franklin_fd630


In [140]:
len(successful_df)

130